# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [ ]:
import glob

import numpy as np
import xarray as xr

from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

# TODO: Update SET_NAME and SET_DIR
SET_NAME = "tropical_subseasonal"
SET_DIR = "886-jjb"

DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{SET_DIR}/{SET_NAME}/wavenumber-frequency/"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/chengzhu/tests/tropical_diags_subsetting/tropical_variability_model_obs_refine/{SET_NAME}/wavenumber-frequency/"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [2]:
def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")

In [3]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace("main", SET_DIR)

        print("Comparing:")
        print(f"    * {fp_dev}")
        print(f"    * {fp_main}")

        ds1 = xr.open_dataset(fp_dev)
        ds2 = xr.open_dataset(fp_main)

        try:
            xr.testing.assert_allclose(
                ds1,
                ds2,
                atol=atol,
                rtol=rtol,
            )
        except (KeyError, AssertionError) as e:
            print(f"    {e}")
        else:
            print(f"    * All close and within relative tolerance ({rtol})")


def _get_var_data(ds: xr.Dataset, var_key: str) -> np.ndarray:
    """Get the variable data using a list of matching keys.

    The `main` branch saves the dataset using the original variable name,
    while the dev branch saves the variable with the derived variable name.
    The dev branch is performing the expected behavior here.

    Parameters
    ----------
    ds : xr.Dataset
        _description_
    var_key : str
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """

    data = None

    var_keys = DERIVED_VARIABLES[var_key].keys()
    var_keys = [var_key] + list(sum(var_keys, ()))

    for key in var_keys:
        if key in ds.data_vars.keys():
            data = ds[key].values
            break

    return data

## 1. Check for matching and equal number of files


In [4]:
_check_if_matching_filecount()

Matching file count (63 and 63).


## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [5]:
_get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/chengzhu/tests/tropical_diags_subsetting/tropical_variability_model_obs_refine/tropical_subseasonal/wavenumber-frequency/FLUT_background_15N-15S_0.nc
    * /global/cfs/cdirs/e3sm/www/chengzhu/tests/tropical_diags_subsetting/tropical_variability_model_obs_refine/tropical_subseasonal/wavenumber-frequency/FLUT_background_15N-15S_0.nc
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/chengzhu/tests/tropical_diags_subsetting/tropical_variability_model_obs_refine/tropical_subseasonal/wavenumber-frequency/FLUT_background_15N-15S_1.nc
    * /global/cfs/cdirs/e3sm/www/chengzhu/tests/tropical_diags_subsetting/tropical_variability_model_obs_refine/tropical_subseasonal/wavenumber-frequency/FLUT_background_15N-15S_1.nc
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/chengzhu/tests/tropical_diags_subsetting/tropical_variability_model_obs_refine/tropical_subsea

### Results

All files are within rtol 1e-5, so the changes should be good to go.
